In [ ]:
import sys
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)

import time

# Add a sleep statement to wait for imports to be fully loaded
time.sleep(20)  # You can adjust the sleep duration based on your needs

from aif360.datasets import GermanDataset
time.sleep(20)
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
time.sleep(20)
from aif360.algorithms.preprocessing import Reweighing
time.sleep(20)

from IPython.display import Markdown, display
time.sleep(20)

# Import matplotlib for visualization
import matplotlib.pyplot as plt

dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

# Additional fairness metrics
display(Markdown("#### Fairness metrics on transformed training dataset"))

# Disparate Impact
disparate_impact = metric_transf_train.disparate_impact()
print("Disparate Impact (favorable label):", disparate_impact)

# Statistical Parity Difference
statistical_parity_difference = metric_transf_train.statistical_parity_difference()
print("Statistical Parity Difference (favorable label):", statistical_parity_difference)

# Create a new dataset by applying the same preprocessing steps as the transformed dataset
dataset_orig_train_pred = RW.transform(dataset_orig_train)

# Classification Metrics
classified_metric_transf_train = ClassificationMetric(dataset_orig_train_pred, 
                                                      dataset_transf_train,
                                                      unprivileged_groups=unprivileged_groups,
                                                      privileged_groups=privileged_groups)

# Equal Opportunity Difference
equal_opportunity_difference = classified_metric_transf_train.equal_opportunity_difference()
print("Equal Opportunity Difference:", equal_opportunity_difference)

# Equalized Odds Difference
equalized_odds_difference = classified_metric_transf_train.equalized_odds_difference()
print("Equalized Odds Difference:", equalized_odds_difference)

# Average Odds Difference
average_odds_difference = classified_metric_transf_train.average_odds_difference()
print("Average Odds Difference:", average_odds_difference)

# Theil Index
theil_index = classified_metric_transf_train.theil_index()
print("Theil Index:", theil_index)

# Predictive Parity
predictive_parity_difference = classified_metric_transf_train.predictive_parity_difference()
print("Predictive Parity Difference:", predictive_parity_difference)

# Define function to plot fairness metrics
def plot_fairness_metrics(disparate_impact, statistical_parity_difference, equal_opportunity_difference, 
                          equalized_odds_difference, average_odds_difference, theil_index, 
                          predictive_parity_difference):
    labels = ['Disparate Impact', 'Statistical Parity Difference', 'Equal Opportunity Difference', 
              'Equalized Odds Difference', 'Average Odds Difference', 'Theil Index', 
              'Predictive Parity Difference']
    metrics = [disparate_impact, statistical_parity_difference, equal_opportunity_difference, 
               equalized_odds_difference, average_odds_difference, theil_index, 
               predictive_parity_difference]

    plt.figure(figsize=(12, 8))
    plt.barh(labels, metrics, color='skyblue')
    plt.title('Fairness Metrics')
    plt.xlabel('Value')
    plt.xlim(0, max(metrics) * 1.2)  # Adjust xlim for better visualization
    plt.show()

# Plot fairness metrics
plot_fairness_metrics(disparate_impact, statistical_parity_difference, equal_opportunity_difference, 
                      equalized_odds_difference, average_odds_difference, theil_index, 
                      predictive_parity_difference)
